<a href="https://colab.research.google.com/github/dev-ansh-r/devanshrep/blob/master/cudapy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cuda call

In [ ]:
import cupy as cp
import numpy as np

In [ ]:
array_cpu = np.random.randint(0,255, size=(2000,2000))
array_cpu.nbytes/1e6

32.0

In [ ]:
array_gpu = cp.asarray(array_cpu)

In [ ]:
%%time

cp.asarray(array_cpu)   

CPU times: user 7.85 ms, sys: 640 µs, total: 8.49 ms
Wall time: 9.36 ms


array([[ 63,  56, 231, ...,  64, 227,   7],
       [252, 246, 217, ..., 240, 213, 155],
       [190, 173, 108, ...,  97, 115, 152],
       ...,
       [238,  10, 155, ..., 106,   0, 183],
       [ 50,  88, 231, ..., 108, 150,  44],
       [188, 140, 208, ...,  69, 139,  33]])

In [ ]:
from scipy import fft

In [ ]:
%%time

fft.fftn(array_cpu)

In [ ]:
fft.fftn(array_gpu)

In [ ]:
from cupyx.scipy import fft as fft_gpu

In [ ]:
%%time

fft_gpu.fftn(array_gpu)

In [ ]:

fft_cpu = fft.fftn(array_cpu)
fft_sent_back = cp.asnumpy(fft_gpu.fftn(array_gpu))
np.allclose(fft_sent_back, fft_cpu)

True

In [ ]:
 type(np.max(array_gpu))

cupy._core.core.ndarray

In [ ]:
from numba import cuda


# Add 1 to each element of Matrix

In [ ]:
A = cp.random.randint(0,10, size=(4,4))
A

array([[0, 7, 1, 7],
       [3, 3, 6, 9],
       [7, 1, 5, 1],
       [0, 2, 3, 5]])

In [ ]:
@cuda.jit
def add_kernel(B):
  row,column = cuda.grid(2)
  B[row, column]+=1
  

In [ ]:
threadsperblock = (16, 16)  # each block will contain 16x16 threads, typically 128 - 512 threads/block
blockspergrid_x = int(np.ceil(A.shape[0] / threadsperblock[0]))
blockspergrid_y = int(np.ceil(A.shape[1] / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)  # we calculate the gridsize (number of blocks) from array
print(blockspergrid)
print(f"The kernel will be executed up to element {threadsperblock[0]*blockspergrid_x}")

(1, 1)
The kernel will be executed up to element 16


In [ ]:
add_kernel[blockspergrid,threadsperblock](A)

In [ ]:
A

array([[ 8,  8,  4,  2],
       [ 4, 10,  9,  1],
       [ 4,  8,  5,  4],
       [ 4,  6,  8,  5]])

# Multiplication of 2 Matrix


In [ ]:
cuda.detect()

Found 1 CUDA devices
id 0             b'Tesla T4'                              [SUPPORTED]
                      compute capability: 7.5
                           pci device id: 4
                              pci bus id: 0
Summary:
	1/1 devices are supported


True

In [ ]:
A = cp.random.randint(0,10, size=(4,4))
B = cp.random.randint(0,10, size=(4,4))
C = cp.zeros((2000, 2000), dtype=np.float64)       # array where we store answer 

In [ ]:
@cuda.jit
def matmul(A,B,C):
  row,column = cuda.grid(2)
  if row < C.shape[0] and column < C.shape[1]:
    tmp = 0
    for k in range(A.shape[1]):
      tmp += A[row, k] * B[k, column]
    C[row,column] = tmp


In [ ]:
matmul[blockspergrid,threadsperblock](A,B,C)

In [ ]:
C

array([[57., 62., 55., ...,  0.,  0.,  0.],
       [86., 90., 73., ...,  0.,  0.,  0.],
       [58., 62., 58., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])


# Fast matrix multiplication

In [ ]:
A = cp.random.randint(0,10, size=(4,4))
B = cp.random.randint(0,10, size=(4,4))
C = cp.zeros((2000, 2000), dtype=np.float64)   

In [ ]:
# faster multiplication can be obtained by making use of shared memory between threads in the same block
# this requires more thinking about non-obvious implementation!
from numba import cuda
import cupy as cp
import numpy as np
from numba import float32, int32, float64

# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 16

@cuda.jit
def fast_matmul(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(bpg):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

ModuleNotFoundError: ignored

In [ ]:
SIZE = 200
A = cp.random.uniform(1, 10, size =(SIZE, SIZE), dtype=np.float32)
B = cp.random.uniform(1, 10, size =(SIZE, SIZE), dtype=np.float32)
C_slow = cp.zeros((SIZE,SIZE), dtype=np.float32)
C_fast = cp.zeros((SIZE,SIZE), dtype=np.float32)

In [ ]:
threadsperblock = (TPB, TPB)
blockspergrid = int(np.ceil(SIZE / threadsperblock[0]))
blockspergrid = (blockspergrid, blockspergrid)

In [ ]:
matmul[blockspergrid, threadsperblock](A, B, C_slow)
C_slow

array([[5756.09  , 5971.56  , 5567.109 , ..., 5565.8794, 5920.4844,
        5465.4014],
       [5834.315 , 6096.5225, 5675.827 , ..., 5792.4824, 6177.8813,
        5545.346 ],
       [5978.775 , 6366.777 , 5901.91  , ..., 5885.3276, 6277.2495,
        5984.8447],
       ...,
       [5869.126 , 6291.7935, 5685.1313, ..., 5758.0044, 6030.998 ,
        5575.953 ],
       [6095.325 , 6570.4985, 5869.72  , ..., 5827.381 , 6271.69  ,
        5789.46  ],
       [6005.316 , 6234.4473, 5872.95  , ..., 6030.748 , 6300.7305,
        5864.4854]], dtype=float32)

In [ ]:
%%timeit

fast_matmul[blockspergrid, threadsperblock](A,B,C_fast)
C_fast

NameError: ignored

In [ ]:
cp.allclose(C_slow, C_fast)

array(False)